In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('bid20-21data.txt')
winter2 = pd.read_csv ('bid21-22data.txt')
winter3 = pd.read_csv ('bid22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_26704\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_26704\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_26704\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [6]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_26704\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26704\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26704\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [7]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 56]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 56]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 56]

In [8]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [9]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [10]:
bid = LocationInfo(
    'Block Island', 'Rhode Island', 
    latitude=41.1617,  
    longitude=-71.5843,
    timezone='America/Rhode_Island'
)

bid

LocationInfo(name='Block Island', region='Rhode Island', timezone='America/Rhode_Island', latitude=41.1617, longitude=-71.5843)

In [11]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(bid.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [12]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(bid.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [13]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(bid.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [14]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:53:05  2020-12-01 16:17:54  2020-12-01
1   2020-12-02 06:54:06  2020-12-02 16:17:40  2020-12-02
2   2020-12-03 06:55:06  2020-12-03 16:17:29  2020-12-03
3   2020-12-04 06:56:04  2020-12-04 16:17:20  2020-12-04
4   2020-12-05 06:57:02  2020-12-05 16:17:13  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:28:04  2021-02-24 17:31:28  2021-02-24
86  2021-02-25 06:26:34  2021-02-25 17:32:38  2021-02-25
87  2021-02-26 06:25:04  2021-02-26 17:33:49  2021-02-26
88  2021-02-27 06:23:33  2021-02-27 17:34:59  2021-02-27
89  2021-02-28 06:22:00  2021-02-28 17:36:09  2021-02-28

[90 rows x 3 columns]


In [15]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:52:50  2021-12-01 16:17:57  2021-12-01
1   2021-12-02 06:53:51  2021-12-02 16:17:43  2021-12-02
2   2021-12-03 06:54:51  2021-12-03 16:17:31  2021-12-03
3   2021-12-04 06:55:50  2021-12-04 16:17:22  2021-12-04
4   2021-12-05 06:56:48  2021-12-05 16:17:14  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:28:26  2022-02-24 17:31:10  2022-02-24
86  2022-02-25 06:26:56  2022-02-25 17:32:21  2022-02-25
87  2022-02-26 06:25:26  2022-02-26 17:33:32  2022-02-26
88  2022-02-27 06:23:55  2022-02-27 17:34:42  2022-02-27
89  2022-02-28 06:22:23  2022-02-28 17:35:52  2022-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:52:35  2022-12-01 16:18:01  2022-12-01
1   2022-12-02 06:53:36  2022-12-02 16:17:46  2022-12-02
2   2022-12-03 06:54:37  2022-12-03 16:17:34  2022-12-03
3   2022-12-04 06:55:36  2022-12-04 16:17:23  2022-12-04
4   2022-12-05 06:56:34  2022-12-05 16:17:16  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:28:47  2023-02-24 17:30:53  2023-02-24
86  2023-02-25 06:27:18  2023-02-25 17:32:04  2023-02-25
87  2023-02-26 06:25:48  2023-02-26 17:33:15  2023-02-26
88  2023-02-27 06:24:17  2023-02-27 17:34:25  2023-02-27
89  2023-02-28 06:22:45  2023-02-28 17:35:35  2023-02-28

[90 rows x 3 columns]


In [17]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [18]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,BID,2020-12-16 23:56:00,0.0001,-SN BR,2020-12-16,2020-12-16 07:05:56,2020-12-16 16:18:33
1,BID,2020-12-17 00:56:00,0.0001,-SN BR,2020-12-17,2020-12-17 07:06:35,2020-12-17 16:18:54
2,BID,2020-12-17 01:56:00,0.2500,-SN BR,2020-12-17,2020-12-17 07:06:35,2020-12-17 16:18:54
3,BID,2020-12-17 02:56:00,0.0001,-SN BR,2020-12-17,2020-12-17 07:06:35,2020-12-17 16:18:54
4,BID,2020-12-17 03:56:00,0.7600,-SN BR,2020-12-17,2020-12-17 07:06:35,2020-12-17 16:18:54
5,BID,2021-01-06 01:56:00,0.0001,-SN,2021-01-06,2021-01-06 07:12:03,2021-01-06 16:32:53
6,BID,2021-01-06 05:56:00,0.0001,-SN,2021-01-06,2021-01-06 07:12:03,2021-01-06 16:32:53
7,BID,2021-01-26 19:56:00,0.0001,-SN,2021-01-26,2021-01-26 07:02:41,2021-01-26 16:55:40
8,BID,2021-01-26 20:56:00,0.0001,-SN BR,2021-01-26,2021-01-26 07:02:41,2021-01-26 16:55:40
9,BID,2021-01-27 00:56:00,1.5200,-SN BR,2021-01-27,2021-01-27 07:01:51,2021-01-27 16:56:55


In [19]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,BID,2021-12-09 06:56:00,0.0001,-SN,2021-12-09,2021-12-09 07:00:25,2021-12-09 16:17:09
1,BID,2021-12-09 08:56:00,0.0001,-SN,2021-12-09,2021-12-09 07:00:25,2021-12-09 16:17:09
2,BID,2021-12-24 05:56:00,0.0001,-SN,2021-12-24,2021-12-24 07:10:03,2021-12-24 16:22:13
3,BID,2021-12-24 06:56:00,0.0001,-SN,2021-12-24,2021-12-24 07:10:03,2021-12-24 16:22:13
4,BID,2021-12-24 07:56:00,0.0001,-SN BR,2021-12-24,2021-12-24 07:10:03,2021-12-24 16:22:13
...,...,...,...,...,...,...,...
104,BID,2022-02-15 01:56:00,0.0001,-SN,2022-02-15,2022-02-15 06:41:08,2022-02-15 17:20:20
105,BID,2022-02-19 19:56:00,0.0001,-SN BR,2022-02-19,2022-02-19 06:35:40,2022-02-19 17:25:12
106,BID,2022-02-25 07:56:00,0.7600,-SN BR,2022-02-25,2022-02-25 06:26:56,2022-02-25 17:32:21
107,BID,2022-02-25 08:56:00,0.7600,-SN BR,2022-02-25,2022-02-25 06:26:56,2022-02-25 17:32:21


In [20]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,BID,2022-12-12 07:56:00,0.0001,-SN,2022-12-12,2022-12-12 07:02:42,2022-12-12 16:17:26
1,BID,2022-12-12 08:56:00,0.0001,-SN BR,2022-12-12,2022-12-12 07:02:42,2022-12-12 16:17:26
2,BID,2022-12-12 09:56:00,0.0001,-SN BR,2022-12-12,2022-12-12 07:02:42,2022-12-12 16:17:26
3,BID,2022-12-14 09:56:00,0.0001,-SN,2022-12-14,2022-12-14 07:04:13,2022-12-14 16:17:50
4,BID,2022-12-24 03:56:00,0.0001,-SN,2022-12-24,2022-12-24 07:09:57,2022-12-24 16:22:04
5,BID,2022-12-24 04:56:00,0.0001,-SN,2022-12-24,2022-12-24 07:09:57,2022-12-24 16:22:04
6,BID,2022-12-24 05:56:00,0.0001,-SN BR,2022-12-24,2022-12-24 07:09:57,2022-12-24 16:22:04
7,BID,2022-12-24 06:56:00,0.0001,-SN BR,2022-12-24,2022-12-24 07:09:57,2022-12-24 16:22:04
8,BID,2022-12-24 07:56:00,0.0001,SN FZFG,2022-12-24,2022-12-24 07:09:57,2022-12-24 16:22:04
9,BID,2022-12-24 08:56:00,0.0001,-SN BR,2022-12-24,2022-12-24 07:09:57,2022-12-24 16:22:04


In [21]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
10,BID,2021-01-28 16:56:00,0.0001,-SN,2021-01-28,2021-01-28 07:00:59,2021-01-28 16:58:10
12,BID,2021-02-01 12:56:00,0.0001,-SN BR,2021-02-01,2021-02-01 06:57:15,2021-02-01 17:03:10
13,BID,2021-02-01 13:56:00,0.2500,-SN BR,2021-02-01,2021-02-01 06:57:15,2021-02-01 17:03:10
14,BID,2021-02-01 14:56:00,0.2500,-SN BR,2021-02-01,2021-02-01 06:57:15,2021-02-01 17:03:10
15,BID,2021-02-01 15:56:00,0.5100,-SN BR,2021-02-01,2021-02-01 06:57:15,2021-02-01 17:03:10
16,BID,2021-02-01 16:56:00,0.5100,-SN BR,2021-02-01,2021-02-01 06:57:15,2021-02-01 17:03:10
21,BID,2021-02-03 10:56:00,0.0001,-SN,2021-02-03,2021-02-03 06:55:12,2021-02-03 17:05:41
22,BID,2021-02-03 11:56:00,0.0001,-SN,2021-02-03,2021-02-03 06:55:12,2021-02-03 17:05:41
23,BID,2021-02-07 15:56:00,1.0200,-SN BR,2021-02-07,2021-02-07 06:50:47,2021-02-07 17:10:42
24,BID,2021-02-07 16:56:00,1.2700,-SN BR,2021-02-07,2021-02-07 06:50:47,2021-02-07 17:10:42


In [22]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_26704\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26704\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26704\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [23]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
12,BID,2021-02-01 12:56:00,0.0001,-SN BR,2021-02-01,2021-02-01 06:57:15,2021-02-01 17:03:10,2021-02-01 07:57:15,2021-02-01 16:03:10
13,BID,2021-02-01 13:56:00,0.2500,-SN BR,2021-02-01,2021-02-01 06:57:15,2021-02-01 17:03:10,2021-02-01 07:57:15,2021-02-01 16:03:10
14,BID,2021-02-01 14:56:00,0.2500,-SN BR,2021-02-01,2021-02-01 06:57:15,2021-02-01 17:03:10,2021-02-01 07:57:15,2021-02-01 16:03:10
15,BID,2021-02-01 15:56:00,0.5100,-SN BR,2021-02-01,2021-02-01 06:57:15,2021-02-01 17:03:10,2021-02-01 07:57:15,2021-02-01 16:03:10
21,BID,2021-02-03 10:56:00,0.0001,-SN,2021-02-03,2021-02-03 06:55:12,2021-02-03 17:05:41,2021-02-03 07:55:12,2021-02-03 16:05:41
22,BID,2021-02-03 11:56:00,0.0001,-SN,2021-02-03,2021-02-03 06:55:12,2021-02-03 17:05:41,2021-02-03 07:55:12,2021-02-03 16:05:41
23,BID,2021-02-07 15:56:00,1.0200,-SN BR,2021-02-07,2021-02-07 06:50:47,2021-02-07 17:10:42,2021-02-07 07:50:47,2021-02-07 16:10:42
32,BID,2021-02-11 07:56:00,0.0001,-SN BR,2021-02-11,2021-02-11 06:45:58,2021-02-11 17:15:42,2021-02-11 07:45:58,2021-02-11 16:15:42
33,BID,2021-02-11 08:56:00,0.0001,-SN BR,2021-02-11,2021-02-11 06:45:58,2021-02-11 17:15:42,2021-02-11 07:45:58,2021-02-11 16:15:42
34,BID,2021-02-11 09:56:00,0.2500,-SN BR,2021-02-11,2021-02-11 06:45:58,2021-02-11 17:15:42,2021-02-11 07:45:58,2021-02-11 16:15:42


In [24]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [25]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [26]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.14888823529411765
2021-2022: 0.11033846153846155
2022-2023: 0.07246666666666665


In [27]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,BID,2020-12-16 23:56:00,0.0001,-SN BR,2020-12-16,2020-12-16 07:05:56,2020-12-16 16:18:33
1,BID,2020-12-17 00:56:00,0.0001,-SN BR,2020-12-17,2020-12-17 07:06:35,2020-12-17 16:18:54
2,BID,2020-12-17 01:56:00,0.2500,-SN BR,2020-12-17,2020-12-17 07:06:35,2020-12-17 16:18:54
3,BID,2020-12-17 02:56:00,0.0001,-SN BR,2020-12-17,2020-12-17 07:06:35,2020-12-17 16:18:54
4,BID,2020-12-17 03:56:00,0.7600,-SN BR,2020-12-17,2020-12-17 07:06:35,2020-12-17 16:18:54
5,BID,2021-01-06 01:56:00,0.0001,-SN,2021-01-06,2021-01-06 07:12:03,2021-01-06 16:32:53
6,BID,2021-01-06 05:56:00,0.0001,-SN,2021-01-06,2021-01-06 07:12:03,2021-01-06 16:32:53
7,BID,2021-01-26 19:56:00,0.0001,-SN,2021-01-26,2021-01-26 07:02:41,2021-01-26 16:55:40
8,BID,2021-01-26 20:56:00,0.0001,-SN BR,2021-01-26,2021-01-26 07:02:41,2021-01-26 16:55:40
9,BID,2021-01-27 00:56:00,1.5200,-SN BR,2021-01-27,2021-01-27 07:01:51,2021-01-27 16:56:55


In [28]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_26704\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26704\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26704\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [29]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [30]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [31]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.5760266666666667
2021-2022: 0.060080952380952374
2022-2023: 0.027866666666666668
